## Import Dependencies  

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

## Preparation
Loading data from **MNist** dataset, and define image embeddings \\

In this case, we have two separated datasets, the unpartitioned and partitioned. We perform traditional training on unpartitioned dataset and perform federated learning on partitioned dataset. The partitioned dataset would be splitted into 10 parts.

In [2]:
# Define transformations to apply to the data
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the pixel values to range [-1, 1]
])

# Load the MNIST dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Define data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)


## Define Model
Define the basic CNN model

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Non-partition

In [4]:
# Train function
def train(model, trainloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')

# Test function
def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy on test set: {100 * correct / total}%')

In [5]:
# Train non-partitioned model
non_partitioned_model = CNN()
non_partitioned_optimizer = optim.Adam(non_partitioned_model.parameters(), lr=0.001)
non_partitioned_criterion = nn.CrossEntropyLoss()
print("Training non-partitioned model...")
train(non_partitioned_model, trainloader, non_partitioned_criterion, non_partitioned_optimizer)
print("Testing non-partitioned model...")
test(non_partitioned_model, testloader)

Training non-partitioned model...
Epoch 1, Loss: 0.12053956443245212
Epoch 2, Loss: 0.04296070709419437
Epoch 3, Loss: 0.029533452758398682
Epoch 4, Loss: 0.020725786873478987
Epoch 5, Loss: 0.016609743203362934
Testing non-partitioned model...
Accuracy on test set: 99.08%


### Partitioned

In [8]:
# Train partitioned model (simulate federated learning)
partition_size = len(trainset) // 10
data_partitions = [torch.utils.data.Subset(trainset, range(i * partition_size, (i + 1) * partition_size))
                   for i in range(10)]

partitioned_models = []
partitioned_optimizers = []
for _ in range(10):
    model = CNN()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    print("Training partitioned model...")
    train(model, torch.utils.data.DataLoader(data_partitions[_], batch_size=32, shuffle=True), criterion, optimizer)
    partitioned_models.append(model)
    partitioned_optimizers.append(optimizer)

# Aggregate model updates
print("Aggregating model updates...")
for i in range(1, 10):
    for params_source, params_target in zip(partitioned_models[i].parameters(), partitioned_models[0].parameters()):
        params_target.data += params_source.data

# Average aggregated model parameters
for params_target in partitioned_models[0].parameters():
    params_target.data /= 10

aggregated_model = partitioned_models[0]

Training partitioned model...
Epoch 1, Loss: 0.5389743443935159
Epoch 2, Loss: 0.12819059653098674
Epoch 3, Loss: 0.08596511012403929
Epoch 4, Loss: 0.061944813763554346
Epoch 5, Loss: 0.04039334639797899
Training partitioned model...
Epoch 1, Loss: 0.5271242829535078
Epoch 2, Loss: 0.13601120816484252
Epoch 3, Loss: 0.08528326340812317
Epoch 4, Loss: 0.058867816891932385
Epoch 5, Loss: 0.046563847318967864
Training partitioned model...
Epoch 1, Loss: 0.5548746767552927
Epoch 2, Loss: 0.1298850610545103
Epoch 3, Loss: 0.08472613878226146
Epoch 4, Loss: 0.04199276243007068
Epoch 5, Loss: 0.036628803658730134
Training partitioned model...
Epoch 1, Loss: 0.5086175176572609
Epoch 2, Loss: 0.12552474394559543
Epoch 3, Loss: 0.07053496785947379
Epoch 4, Loss: 0.05023680604620282
Epoch 5, Loss: 0.029827385666957272
Training partitioned model...
Epoch 1, Loss: 0.52920163478306
Epoch 2, Loss: 0.11591915913827797
Epoch 3, Loss: 0.07385448826863332
Epoch 4, Loss: 0.04916729882825166
Epoch 5, Loss

In [9]:
# Test federated model
print("Testing aggregated model...")
test(aggregated_model, testloader)

Testing aggregated model...
Accuracy on test set: 11.35%
